In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from category_encoders import OneHotEncoder

import pickle as pkl
import pandas as pd
import os
import numpy as np

In [20]:
df = pd.read_csv(os.path.join('..', 'data', 'filter-data-cleaned.csv'))
df = df.drop(columns=['company_name', 'quote_id', 'description', 'qty', 'dimensions', 'item_type', 'Depth', 'Pockets', 'Date'])

In [21]:
df.head()

,unit_price,filter_efficiency,Length,Height,Gutter
0,7.5,G4,625.0,500.0,50.0
1,27.5,F7,625.0,500.0,50.0
2,28.5,F7,592.0,490.0,48.0
3,27.5,F7,592.0,592.0,45.0
4,17.5,F7,287.0,592.0,45.0


## Keep only the M5 filters

In [22]:
df = df[df['filter_efficiency'] == 'M5']

## Train-Test split

In [23]:
X = df.drop(columns=['unit_price'])
y = df['unit_price']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [25]:
ohe = OneHotEncoder(cols=['filter_efficiency'], use_cat_names=True)
X_train = ohe.fit_transform(X_train)
X_test = ohe.transform(X_test)

## Train the best model for all filters

In [26]:
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train.values, y_train.values)

y_pred = model.predict(X_test.values)

print(f'{np.mean(np.abs(y_pred - y_test.values) / y_test.values) * 100:.2f}%')

37.22%


## Save the model

In [27]:
pkl.dump(model, open(os.path.join('..', 'models', 'best_model_m5.pkl'), 'wb'))